In [3]:
# first step
# Version Check!
!python --version

Python 3.8.18


In [4]:
# I hate warning message!
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# # I like reading book!
# from langchain_community.document_loaders import PyMuPDFLoader

# # PDF file list generation
# pdf_files = [f"./data/data{i}.pdf" for i in range(1,10)]

# # empty list geneation
# all_pages = []

# # Load each PDF file and add list
# for pdf_file in pdf_files:
#     loader = PyMuPDFLoader(pdf_file)
#     pages = loader.load()
#     all_pages.extend(pages)


from langchain_community.document_loaders import TextLoader
import glob

directory_path = "./dataset/preprocessed_text/*.txt" 

# 모든 텍스트 파일 경로 가져오기
file_paths = glob.glob(directory_path)
all_pages = []

for file_path in file_paths:
    loader = TextLoader(file_path, encoding='utf-8')
    data = loader.load()
    all_pages.extend(data)



In [4]:
# Document seperate sentence

# characterTextSplitter
#RecursiveCharcterTextSplitter
#TokenTextSplitter
#SemanticChunker

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(all_pages)

# 결과 확인
print(f"총 {len(docs)}개의 청크로 분리되었습니다.")

총 1001개의 청크로 분리되었습니다.


In [5]:
# check chunk
docs[5]

Document(page_content='강사/기관 찾기 > 교육기관 찾기\n\n신청방법\n\n3. 직접교육(자체교육) 실시\n\n사업주 및 근로자가 직접 한국장애인고용공단의 교육 콘텐츠를 활용하여 교육을 실시하실 수 있습니다. 단, 상시근로자가 300인 이상인 사업장체는 자격이 있는 강사(사내강사를 포함) 또는 교육기관을 통해 교육을 실시하셔야 합니다.\n\n콘텐츠 확인\n\n직장 내 장애인 인식개선 교육 포털(edu.kead.or.kr) > 자료실 >\n\n교육자료실(PPT, 동영상, 간이교육자료 등을 활용하여 교육)\n\n실시 가능\n\n미이행 시 과태료 부과', metadata={'source': './dataset/preprocessed_text\\data10_preprocessed_chunk10.txt'})

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
#HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')


embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    # model_kwargs={'device':'cpu'},
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

In [8]:
from langchain.vectorstores import FAISS

MY_FAISS_INDEX = 'MY_FAISS_INDEX'
# 로컬 DB 불러오기
vectorstore = FAISS.load_local(MY_FAISS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True # 잠재적으로 위험한 데이터 구조나 객체를 포함할 수 있는 인덱스 파일의 로딩을 허용. 주로 자신이 직접 생성하고 저장한 인덱스 파일을 로드할 때 사용
                               )


# from langchain.schema import Document
# import os
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Chroma

# docs = [
#     Document(page_content="This is a document.", metadata={"id": "doc1"}),
#     Document(page_content="This is another document.", metadata={"id": "doc2"})
# ]

# # 벡터 저장소 경로 설정
# vectorstore_path = 'vectorstore'
# os.makedirs(vectorstore_path, exist_ok=True)

# # 벡터 저장소 생성 및 저장
# vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# vectorstore.persist()

# print("Vectorstore created and persisted")

In [14]:
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
# model = ChatOllama(model="gemma2", temperature=0)


model = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    temperature=0.5,
    # top_p=0,
    top_k=1,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [19]:
# retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # 유사도 높은 3문장 추출


# 게시글 생성

In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = '''
장애인식 개선을 위한 인스타그램 게시글 생성:
- 우리 사회에서 장애인에 대한 차별을 없애기 위해 장애인식 개선이 중요합니다.
- 이 게시글은 장애인과 비장애인이 함께 어우러지는 사회를 만드는 데 기여하고자 합니다.
- 답변은 한국어로 해줍니다.
- 말투는 공손한 말투를 사용합니다.
- 아래의 <>와 같이 장애인을 이해하기 위해 비장애인이 사전에 알면 도움이 되거나 간단하게 실천할 수 있는 것에 대해서 사용자 입력에게 알려주는 장애 인식 개선을 위한 문장을 생성해주세요
<오늘은 청각장애인에 대해서 알아볼 것인데요. 
청각 장애인은 청력 손실이 심하여 보청기를 착용해도 청각을 통한 의사소통이 불가능 또는 곤란한 상태이거나, 청력이 남아 있어도 보청기를 착용해야 청각을 통한 의사소통이 가능하여 청각에 의한 교육적 성취가 어려운 사람을 말합니다.

청각장애인들이 일상생활에서 어떤 어려움을 겪는지, 그리고 그들이 사회에서 더 잘 적응하고 소통할 수 있도록 돕는 방법들에 대해 자세히 살펴보겠습니다.
기본적인 수어 배우기:

사례: 간단한 인사말이나 기본적인 표현을 수어로 배우면 청각장애인과의 소통에 큰 도움이 됩니다. 예를 들어, '안녕하세요', '감사합니다' 등의 표현을 수어로 익혀두면 청각장애인과의 첫 만남에서 긍정적인 인상을 줄 수 있습니다.
서면으로 소통하기:

사례: 레스토랑이나 카페에서 주문을 받을 때, 메모지와 펜을 준비해 두어 청각장애인이 원하는 주문을 서면으로 작성할 수 있게 합니다. 이를 통해 오해를 줄이고 보다 정확한 의사소통이 가능합니다.
입모양과 표정 사용:

사례: 청각장애인과 대화할 때, 천천히 말하고 입모양과 표정을 활용하면 의사소통이 더 원활해집니다. 예를 들어, 청각장애인이 당신의 말을 읽기 쉽도록 얼굴을 직접 바라보고 명확하게 발음하는 것이 좋습니다.
정보 접근성 향상:

사례: 행사나 모임에서 중요한 정보를 제공할 때, 자막이나 필기된 자료를 함께 제공하여 청각장애인이 놓치지 않도록 합니다. 예를 들어, 프레젠테이션 시 화면에 자막을 추가하거나, 회의록을 미리 공유하는 방법이 있습니다.
청각장애인 지원 캠페인 참여:

사례: 청각장애인에 대한 인식을 높이고 지원하는 다양한 캠페인에 참여하거나 후원할 수 있습니다. 예를 들어, 청각장애인 지원 단체의 활동에 참여하거나 기부를 통해 도움을 줄 수 있습니다.
이러한 작은 노력들이 모여 청각장애인들이 더 나은 환경에서 생활하고 소통할 수 있게 됩니다. 일반인들도 이러한 방법들을 실천함으로써 청각장애인과의 의사소통을 개선하고, 더 나아가 서로를 이해하는 데 큰 도움이 될 것입니다.>


# 여기에 문서에서 검색된 내용을 기반으로 한 추가적인 정보를 포함시킵니다.
{context}

# 해시태그 생성 지시:
# 게시글의 내용을 분석하여 주요 주제와 관련된 해시태그를 제안해주세요.
# 해쉬 태그는 8개 이상 작성되어야 한며 #나란히 # 상생 은 고정적으로 존재한다.
# 예시: 게시글에서 '장애인식 개선'과 '포용 사회'가 주요 주제로 다뤄진다면, 관련 해시태그로 #장애인식개선, #포용사회 등을 제안할 수 있습니다.
# 최종적으로 #나란히 #상생 #장애인식개선 #포용사회 #청각장애인 #수화 #의사소통 #함께하는세상 과 같이 출력되어야 한다.









Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    # 검색된 문서의 내용을 포맷팅하여 제공합니다.
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query1 = "정신 지체 장애인"
answer1 = rag_chain.invoke(query1)

print("Query:", query1)
print("Answer:", answer1)


#나란히 #상생 #장애인식개선 #포용사회 #청각장애인 #수화 #의사소통 #함께하는세상

정신지체, 또는 지적장애라고도 불리는 이 장애는 개인이 일상 생활과 의사결정에서 어려움을 겪게 만드는 인지 기능의 저하를 특징으로 합니다. 이러한 개인들은 비장애인처럼 일을 처리하는 데 더 많은 시간이 필요할 수 있지만, 여전히 자신의 필요와 욕구를 충족시키기 위해 지원과 자원을 필요로 하는 소중한 구성원입니다.

지적장애인을 대할 때 동정과 시혜의 마음으로 다르게 대우하는 것은 차별로 이어질 수 있습니다. 대신에 우리는 그들의 독특한 능력과 강점을 인식하고 존중하며, 그들이 사회 내에서 의미 있는 기여를 할 수 있도록 필요한 지원과 자원을 제공해야 합니다.

이해와 포용을 증진함으로써, 우리는 장애가 있는 사람들과 그렇지 않은 사람들이 함께 어우러지는 사회를 만들 수 있으며, 모든 개인의 존엄성과 가치를 소중히 여길 수 있습니다.Query: 정신 지체 장애인
Answer: #나란히 #상생 #장애인식개선 #포용사회 #청각장애인 #수화 #의사소통 #함께하는세상

정신지체, 또는 지적장애라고도 불리는 이 장애는 개인이 일상 생활과 의사결정에서 어려움을 겪게 만드는 인지 기능의 저하를 특징으로 합니다. 이러한 개인들은 비장애인처럼 일을 처리하는 데 더 많은 시간이 필요할 수 있지만, 여전히 자신의 필요와 욕구를 충족시키기 위해 지원과 자원을 필요로 하는 소중한 구성원입니다.

지적장애인을 대할 때 동정과 시혜의 마음으로 다르게 대우하는 것은 차별로 이어질 수 있습니다. 대신에 우리는 그들의 독특한 능력과 강점을 인식하고 존중하며, 그들이 사회 내에서 의미 있는 기여를 할 수 있도록 필요한 지원과 자원을 제공해야 합니다.

이해와 포용을 증진함으로써, 우리는 장애가 있는 사람들과 그렇지 않은 사람들이 함께 어우러지는 사회를 만들 수 있으며, 모든 개인의 존엄성과 가치를 소중히 여길 수 있습니다.


In [22]:
print(answer1)

#나란히 #상생 #장애인식개선 #포용사회 #청각장애인 #수화 #의사소통 #함께하는세상

정신지체, 또는 지적장애라고도 불리는 이 장애는 개인이 일상 생활과 의사결정에서 어려움을 겪게 만드는 인지 기능의 저하를 특징으로 합니다. 이러한 개인들은 비장애인처럼 일을 처리하는 데 더 많은 시간이 필요할 수 있지만, 여전히 자신의 필요와 욕구를 충족시키기 위해 지원과 자원을 필요로 하는 소중한 구성원입니다.

지적장애인을 대할 때 동정과 시혜의 마음으로 다르게 대우하는 것은 차별로 이어질 수 있습니다. 대신에 우리는 그들의 독특한 능력과 강점을 인식하고 존중하며, 그들이 사회 내에서 의미 있는 기여를 할 수 있도록 필요한 지원과 자원을 제공해야 합니다.

이해와 포용을 증진함으로써, 우리는 장애가 있는 사람들과 그렇지 않은 사람들이 함께 어우러지는 사회를 만들 수 있으며, 모든 개인의 존엄성과 가치를 소중히 여길 수 있습니다.


# 게시글 내용 요약

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = '''
* 너는 제공된 문장의 핵심 내용을 2가지로 요약해서 문장으로 반환해주는 역할을 해
* 요약된 문장에는 다음의 내용이 포함되어 있어야 합니다.
    - 장애인을 이해하기 위해 비장애인이 사전에 알면 도움이 되거나 간단하게 실천할 수 있는 내용이 포함되어야 합니다.
    - 요약된 내용은 아래의 예와 같이 
## 정신 지체 장애인에 대한 이해를 위한 두 가지 요약:

1. 정신 지체 장애는 학습, 인지 및 사회적 기술 발달이 일반 아동보다 느리거나 어려움을 겪는 상태로, 유전적인 요인, 선천적 질환 등 다양한 원인으로 발생합니다.
2. 정신 지체 장애인과 소통할 때 간단하고 천천히 말하며 눈맞춤과 얼굴 표정을 통해 감정을 전달하는 것이 효과적입니다.
1. 2. 정렬해주어야 합니다.

# 여기에 문서에서 검색된 내용을 기반으로 한 추가적인 정보를 포함시킵니다.
{context}


Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    # 검색된 문서의 내용을 포맷팅하여 제공합니다.
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query2 = answer1
answer2 = rag_chain.invoke(query2)

print("Query:", query2)
print("Answer:", answer2)


Query: ##  정신 지체 장애인에 대해 알아볼까요?

오늘은 정신 지체 장애인에 대해서 알아보겠습니다. 🧠

**정신 지체 장애는 학습, 인지 및 사회적 기술 발달이 일반 아동보다 느리거나 어려움을 겪는 상태를 말합니다.**  이는 유전적인 요인, 선천적 질환, 출산 과정의 문제 등 다양한 원인으로 발생할 수 있습니다.

**정신 지체 장애인들이 일상생활에서 어떤 어려움을 겪는지, 그리고 그들이 사회에서 더 잘 적응하고 소통할 수 있도록 돕는 방법들에 대해 자세히 살펴보겠습니다.**

<정신 지체 장애인과의 소통 시, 간단한 문장으로 천천히 말하는 것이 중요합니다. 또한, 눈을 마주치고 얼굴 표정을 통해 감정을 전달하면 더욱 효과적인 소통이 가능합니다.> 


**#나란히 #상생 #장애인식개선 #포용사회 #정신지체장애인 #소통 #공감 #함께하는세상**  

Answer: ## 정신 지체 장애인에 대한 이해를 위한 두 가지 요약:

1. 정신 지체 장애는 유전적 요인, 선천적 질환 등 다양한 원인으로 발생하며 학습, 인지 및 사회적 기술 발달이 일반 아동보다 느리거나 어려움을 겪는 상태를 말합니다.
2. 정신 지체 장애인과 소통할 때 간단하고 천천히 말하며 눈맞춤과 얼굴 표정을 통해 감정을 전달하는 것이 효과적입니다.  





# 정규 표현식을 이용해서 텍스트 추출

In [16]:
import re
# 정규표현식 패턴
pattern = r'\d+\. [^\d]+'

# 정규표현식으로 문장 추출
matches = re.findall(pattern, answer2)

# 결과 출력
print(matches)

['1. 정신 지체 장애는 유전적 요인, 선천적 질환 등 다양한 원인으로 발생하며 학습, 인지 및 사회적 기술 발달이 일반 아동보다 느리거나 어려움을 겪는 상태를 말합니다.\n', '2. 정신 지체 장애인과 소통할 때 간단하고 천천히 말하며 눈맞춤과 얼굴 표정을 통해 감정을 전달하는 것이 효과적입니다.  \n\n\n']


In [17]:
matches[0]

'1. 정신 지체 장애는 유전적 요인, 선천적 질환 등 다양한 원인으로 발생하며 학습, 인지 및 사회적 기술 발달이 일반 아동보다 느리거나 어려움을 겪는 상태를 말합니다.\n'

In [18]:
matches[1]

'2. 정신 지체 장애인과 소통할 때 간단하고 천천히 말하며 눈맞춤과 얼굴 표정을 통해 감정을 전달하는 것이 효과적입니다.  \n\n\n'

# 이미지 생성

In [ ]:
""" 생성 """
import openai 
import webbrowser
import os

# Replace YOUR_API_KEY with your OpenAI API key
client = openai.OpenAI(api_key = "sk-")

# Call the API

# 1장 생성 시 0.03$ 
response1 = client.images.generate(
  model="dall-e-3",
  prompt=query1+ matches[0] ",애니메이션 풍으로 그려줘",
  size="1024x1024",
  quality="standard",
  n=1,
)

# Show the result that has been pushed to an url
webbrowser.open(response1.data[0].url)

In [ ]:
response1.data[0].url

In [ ]:
""" 생성 """
import openai 
import webbrowser
import os

# Replace YOUR_API_KEY with your OpenAI API key
client = openai.OpenAI(api_key = "sk-")

# Call the API

# 1장 생성 시 0.03$ 
response2 = client.images.generate(
  model="dall-e-3",
  prompt=query1+ matches[1] ",애니메이션 풍으로 그려줘",
  size="1024x1024",
  quality="standard",
  n=1,
)

# Show the result that has been pushed to an url
webbrowser.open(response2.data[0].url)

In [ ]:
response2.data[0].url

# 퀴즈 관리자

In [20]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = '''
* 너는 인스타그램상에서 장애 인식 개선 프로젝트의 교육을 담당하는 조교야
* 너의 일은 입력된 내용과 관련된 <동행 퀴즈>를 관리 생성하는 역할이야
* 너는 공손한 말투를 사용해야 해

장애 바로알기및 장애인식 개선을 위한 <동행 퀴즈>는 다음의 목표를 갖습니다.
- 우리 사회에서 장애인에 대한 차별을 없애기 위해 장애인식 개선
- 이 퀴즈는 장애인과 비장애인이 함께 어우러지는 사회를 만드는 데 기여
- 이 퀴즈는 장애인에 대한 잘못된 상식을 바로잡기
- 퀴즈는 다음과 같은 형식으로 내줘


    안녕하세요! 오늘은 장애인에 대한 이해를 높이는 시간을 가져보도록 하겠습니다.  

    **<동행 퀴즈>** 를 통해 함께 배우고 생각해 볼 수 있도록 준비했습니다. 

    **질문:** 청각장애인에게 전달하고 싶은 말이 있을 때 가장 적절한 방법은 무엇일까요?

    A. 큰 소리로 말한다.
    B. 손짓과 표정으로 의사를 전달한다.
    C. 귀에 닿지 않도록 조용히 말한다.
    D. 글자를 쓰고 보여준다.


**답변을 고민해보세요!**  

# 여기에 문서에서 검색된 내용을 기반으로 한 추가적인 정보를 포함시킵니다.
{context}


Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    # 검색된 문서의 내용을 포맷팅하여 제공합니다.
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query3 = query1 # 게시글 생성시 결정된 topic에 의해 문제 생성
answer3 = rag_chain.invoke(query3)

print("Query:", query3)
print("Answer:", answer3)


Query: 정신 지체 장애인
Answer: 안녕하세요! 오늘은 장애인에 대한 이해를 높이는 시간을 가져보도록 하겠습니다.  

**<동행 퀴즈>** 를 통해 함께 배우고 생각해 볼 수 있도록 준비했습니다. 

**질문:** 정신 지체 장애인과 대화할 때 가장 중요한 것은 무엇일까요?

A. 간단하고 명확하게 말한다.
B. 어려운 단어를 사용해서 설명한다.
C.  그들의 의견을 존중하지 않는다.
D. 끊임없이 질문하며 그들의 지능을 시험한다.


**답변을 고민해보세요!**  

# 여기에 문서에서 검색된 내용을 기반으로 한 추가적인 정보를 포함시킵니다. 



정신 지체 장애는 학습, 언어, 사회적 상호 작용 등에 어려움을 겪는 것을 의미합니다. 정신 지체 장애인과 대화할 때는 **간단하고 명확하게 말하는 것이 중요**하며, 그들의 의견을 존중하고 적극적으로 경청해야 합니다.  



# 퀴즈 정답 관리자

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = '''
* 너는 인스타그램상에서 장애 인식 개선 프로젝트의 댓글을 담당하는 조교야
* 너의 일은 <동행 퀴즈>의 정답과 해설을 알려주는 일이야
* 너는 공손한 말투를 사용해야 해

장애 바로알기및 장애인식 개선을 위한 <동행 퀴즈>는 다음의 목표를 갖습니다.
- 우리 사회에서 장애인에 대한 차별을 없애기 위해 장애인식 개선
- 이 퀴즈는 장애인과 비장애인이 함께 어우러지는 사회를 만드는 데 기여
- 이 퀴즈는 장애인에 대한 잘못된 상식을 바로잡기
- 퀴즈 정답의 형식입니다.
##  시각장애인 안내 퀴즈 정답 및 해설! 🤔

**정답은 C번입니다.** 😊 시각장애인의 팔꿈치를 가볍게 잡고 안내하는 것이 가장 적절합니다. 

**A번과 B번은 오답입니다.**  
* A번처럼 팔을 잡아끌면 시각장애인이 불안하거나 방향 감각을 잃을 수 있습니다. 🚶‍♀️🚶‍♂️
* B번처럼 스스로 길을 찾도록 하는 것은 안전하지 않고, 시각장애인에게 어려움을 줄 수 있습니다. 

**D번은 부분적으로 맞습니다.** 큰 소리로 방향을 지시하는 것도 도움이 될 수 있지만,  직접적인 접촉과 함께 안내하는 것이 더욱 효과적입니다. 🗣️


## 시각장애인 안내 팁!

* **"안녕하세요, 저는 당신을 안내해드릴게요."** 와 같이 말하며 시작하여 시각장애인에게 자신을 소개합니다.
* **시각장애인의 속도에 맞춰 천천히 걸어가세요.** 🐢
* **길을 돌거나 계단을 오르내릴 때는 미리 알려주세요.**  🗣️

**함께 안전하고 편안한 환경을 만들 수 있도록 노력합시다!** 💖
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

def get_instagram_posts():
    # 인스타그램 게시글을 가져오는 함수
    return answer1

def get_quiz_and_discussion_content():
    # 퀴즈와 내용을 가져오는 함수
    quiz_content = answer3
    return quiz_content

# RAG Chain 연결
rag_chain = (
    {
        'context': retriever | format_docs | (lambda context: f"{context}\n\n{get_instagram_posts()}\n\n{get_quiz_and_discussion_content()}"), 
        'question': RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query4 = "정답 공개합니다."
answer4 = rag_chain.invoke(query4)

print("Query:", query4)
print("Answer:", answer4)


Query: 정답 공개합니다.
Answer: 정답은 **A번**입니다! 😊 간단하고 명확한 표현으로 대화하면 정신 지체 장애인도 이해하기 쉽고, 편안하게 소통할 수 있습니다.

B, C, D번은 모두 오답입니다.  

* B번처럼 어려운 단어를 사용하면 오히려 이해가 어렵게 되고, 
* C번처럼 의견을 존중하지 않는 태도는 정신 지체 장애인의 자존감을 낮추고 소통을 방해합니다. 
* D번처럼 지능을 시험하는 질문은 불필요한 스트레스를 주고, 관계 형성에 도움이 되지 않습니다.


**정신 지체 장애인과의 소통 팁!**

* **눈맞춤**: 눈을 마주치면서 대화하면 진심으로 경청하고 있다는 것을 알 수 있습니다.
* **천천히 말하기**:  간단하고 명확한 문장으로 천천히 말하는 것이 좋습니다.
* **반복 확인**: 이해했는지 확인하기 위해 "이해했나요?" 와 같이 물어보세요.
* **긍정적인 피드백**: 작은 노력에도 칭찬과 격려를 아끼지 마세요!


함께 편안하고 의미있는 대화를 나눌 수 있도록 노력합시다! 💖


# 퀴즈 댓글 관리자

In [29]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Prompt 템플릿 생성
template = '''
* 너는 인스타그램상에서 장애 인식 개선 프로젝트의 댓글을 담당하는 조교야
* 너의 일은 <동행 퀴즈>에 대한 사용자 입력의 오답여부와 코멘트를 남겨주는 일이야.
* 너는 공손한 말투를 사용해야 해

장애 바로알기및 장애인식 개선을 위한 <동행 퀴즈>는 다음의 목표를 갖습니다.
- 우리 사회에서 장애인에 대한 차별을 없애기 위해 장애인식 개선
- 이 퀴즈는 장애인과 비장애인이 함께 어우러지는 사회를 만드는 데 기여
- 이 퀴즈는 장애인에 대한 잘못된 상식을 바로잡기
- 출력 형식은 다음과 같습니다.
아쉽게도 B는 정답이 아닙니다😰 

정신 지체 장애인과 대화할 때는 **간단하고 명확하게 말하는 것이** 가장 중요합니다. 어려운 단어를 사용하면 이해하기 어렵고, 오히려 불편함을 느낄 수 있습니다.  
정답은 A번입니다! 😊 간단하고 명확한 표현으로 대화하면 정신 지체 장애인도 이해하기 쉽고, 편안하게 소통할 수 있습니다.


# 여기에 문서에서 검색된 내용을 기반으로 한 추가적인 정보를 포함시킵니다.
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

def get_instagram_posts():
    # 인스타그램 게시글을 가져오는 함수
    return answer1

def get_quiz_and_discussion_content():
    # 퀴즈와 내용을 가져오는 함수
    quiz_content = answer3
    return quiz_content 

# RAG Chain 연결
rag_chain = (
    {
        'context': retriever | format_docs | (lambda context: f"{context}\n\n{get_instagram_posts()}\n\n{get_quiz_and_discussion_content()}"), 
        'question': RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query4 = "지체 장애인" # 인스타그램 그래프 api로 사용자 입력 text 추출
answer4 = rag_chain.invoke(query4)

print("Query:", query4)
print("Answer:", answer4)


Query: 지체 장애인
Answer: 아쉽게도 B는 정답이 아닙니다😰 

정신 지체 장애인과 대화할 때는 **간단하고 명확하게 말하는 것이** 가장 중요합니다. 어려운 단어를 사용하면 이해하기 어렵고, 오히려 불편함을 느낄 수 있습니다.  
정답은 A번입니다! 😊 간단하고 명확한 표현으로 대화하면 정신 지체 장애인도 이해하기 쉽고, 편안하게 소통할 수 있습니다. 





In [ ]:
Query: 지체 장애인
Answer: 아쉽게도 B는 정답이 아닙니다😰 

정신 지체 장애인과 대화할 때는 **간단하고 명확하게 말하는 것이** 가장 중요합니다. 어려운 단어를 사용하면 이해하기 어렵고, 오히려 불편함을 느낄 수 있습니다.  
정답은 A번입니다! 😊 간단하고 명확한 표현으로 대화하면 정신 지체 장애인도 이해하기 쉽고, 편안하게 소통할 수 있습니다. 